In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
#Imports
from google import genai
from tavily import TavilyClient
import os
import re
import json
import sqlite3
from datetime import datetime
from typing import List, Dict, Any, Optional
import numpy as np
from pathlib import Path

In [21]:
# Create client
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
model="gemini-2.5-pro"
tavily_client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) # Initialize Tavily client

# Memory

from memory_system import (
    MemoryStore, MemoryRetriever, FeedbackHandler, 
    ContextBuilder, AgentMemoryIntegration
)

# Initialize memory system
memory_store = MemoryStore("pm_agent_memory.db")
retriever = MemoryRetriever(memory_store)
feedback_handler = FeedbackHandler(memory_store)
context_builder = ContextBuilder(retriever)
memory_integration = AgentMemoryIntegration(
    memory_store, retriever, feedback_handler, context_builder
)

In [24]:
#Agents
class ProductManagerAgent:
    """Product Manager Agent with conversation memory and long-term learning."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.system_instruction = """You are an experienced Product Manager with expertise in:
        - Product strategy and roadmap planning
        - Market research and competitive analysis
        - User research and requirements gathering
        - Feature prioritization and backlog management
        - Stakeholder communication
        - Metrics and KPI definition
        - Go-to-market strategies
        - Agile methodologies
        - Providing feedback to your product specialist, engineering, design, data science, testing, and marketing teams

        You provide practical, actionable advice and ask clarifying questions when needed.
        When provided with search results or past memories, incorporate relevant information into your response and cite sources."""
    
    def __call__(self, user_query):
        """
        Make the agent callable like a function.
        Returns (response, memory_id) tuple.
        """
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(user_query, "pm_interaction")

        # Decide if search is needed
        decision_prompt = f"""Analyze this query and determine if you need web search.

Query: {user_query}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        # Perform search if needed
        search_context = ""
        if needs_search:
            print("🔍 Searching the web...")
            search_results = search_web(user_query)
            if isinstance(search_results, list):
                context = "\n\nWeb Search Results:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious conversation:\n"
            for entry in self.conversation_history[-5:]:  # Keep last 5 exchanges
                conversation_context += f"User: {entry['user']}\nAssistant: {entry['assistant']}\n\n"
        
        # Combine all contexts
        full_query = (
            memory_context +
            conversation_context + 
            f"\nCurrent query: {user_query}" +
            search_context
        )
        
        response = self.client.models.generate_content(
            model=self.model,
            contents=full_query,
            config={'system_instruction': self.system_instruction}
        )
        
        # Store in conversation history
        self.conversation_history.append({
            'user': user_query,
            'assistant': response.text
        })
        
        # Log to long-term memory
        memory_id = self.memory_integration.post_run_hook(
            user_query,
            response.text,
            memory_type="pm_interaction",
            metadata={'search_used': needs_search}
        )

        return response.text, memory_id
    
    def apply_feedback(self, memory_id: int, feedback_type: str, 
                      feedback_content: str = ""):
        """Apply feedback to a specific interaction."""
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("Conversation history cleared.")



In [25]:
#Helper functions
def remove_markdown(response_text):
    """Remove markdown formatting and print clean text."""
    # Remove markdown formatting
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', response_text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'`(.+?)`', r'\1', text)  # Code
    text = re.sub(r'#+\s', '', text)  # Headers
    return text
def chat_with_agent(agent, agent_name="Agent"):
    """
    Generic interactive chat interface for any agent.
    
    Args:
        agent: The agent instance (callable) to chat with
        agent_name: Display name for the agent (default: "Agent")
    
    Type 'exit', 'quit', or 'bye' to end the conversation.
    Type 'clear' to clear conversation history.
    """
    print("=" * 60)
    print(f"{agent_name} Chat")
    print("=" * 60)
    print(f"Start chatting with the {agent_name}!")
    print("Type 'exit', 'quit', or 'bye' to end the chat.")
    print("Type 'clear' to clear conversation history.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print(f"\n{agent_name}: Thanks for chatting! Goodbye!")
            break
        
        if user_input.lower() == 'clear':
            if hasattr(agent, 'clear_history'):
                agent.clear_history()
            continue
            
        if not user_input:
            continue
            
        try:
            response = agent(user_input)
            print(f"\n{agent_name}: {remove_markdown(response)}\n")
        except Exception as e:
            print(f"\nError: {e}\n")
            print("Please try again or type 'exit' to quit.\n")
def search_web(query):
    """Search the web using Tavily."""
    try:
        response = tavily_client.search(query, max_results=5)
        return response['results']
    except Exception as e:
        return f"Search error: {e}"



In [26]:
# Create agent instances
pm_agent = ProductManagerAgent(model=model, client=client, memory_integration=memory_integration)

In [29]:
# Test the agent
query = "If you wanted to build a product team of agents, a team that can take a user query, do research, create user stories, designs, create different documents that engineers can use, how would you go about doint it?"
response, memory_id = pm_agent(query)
print(remove_markdown(response))

Of course. This is an excellent question that builds directly on our previous conversations about building a "product team of agents" and using prioritization frameworks like RICE.

Our initial discussion laid out a strong strategic blueprint: a multi-agent system with specialized roles (Orchestrator, Researchers, Strategist, etc.), a phased workflow, and an incremental roadmap ([pm_interaction]).

Now, let's move from that high-level strategy to a more detailed, operational plan. As a PM building this system, I'd focus on three critical areas to bring this concept to life:

1.  Refining the Agent Workflow with a Prioritization Engine.
2.  Designing the "Human-in-the-Loop" Interface as a Core Product Feature.
3.  Defining a Granular Roadmap with Clear Success Metrics.

---

1. Refining the Agent Workflow with a Prioritization Engine

In our last discussion, the Product Strategist Agent was tasked with creating a "list of potential features." Let's make that process more robust by expli

In [28]:
# Test with memory
response, memory_id = pm_agent("What is the RICE framework?")
print(remove_markdown(response))
print(f"\nMemory ID: {memory_id}")

# Apply feedback
pm_agent.apply_feedback(memory_id, 'positive')

Of course. The RICE framework is a classic and highly effective tool in a Product Manager's toolkit for prioritization. Let's break it down from a practical standpoint.

What is the RICE Framework?

At its core, RICE is a simple scoring system designed to help product teams prioritize features, initiatives, and ideas in a more objective, data-informed way. It forces you to think critically about why you're building something and helps you defend your roadmap decisions to stakeholders.

The goal is to move beyond prioritizing based on the "loudest voice in the room" or pure gut instinct.

The acronym RICE stands for:

1.  Reach
2.  Impact
3.  Confidence
4.  Effort

You calculate a score for each feature using the formula:

(Reach x Impact x Confidence) / Effort = RICE Score

Let's dive into each component.

---

Breaking Down the RICE Components

1. Reach
*   The Question: How many people will this feature affect over a specific time period?
*   How to Measure It: You need a real number